In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

In [2]:
#reading the labels
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
#getting encoded labels
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#onehot=OneHotEncoder()
encoder = LabelEncoder()
#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [4]:
#defining a dict of encoded labels
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [5]:
#loading the traing_images
X = []
Y = []
train_label = shuffle(train_label,random_state=0)
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)
            

X = np.array(X, np.float16)/255.0

100%|██████████| 40479/40479 [02:12<00:00, 304.73it/s]


In [6]:
#splitting
from sklearn.model_selection import train_test_split

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [7]:
#deleting uneccessary datas
import gc
del X, Y
gc.collect()

20

In [8]:
import gc
gc.collect()

20

In [9]:
import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [10]:
# base model. Feel free to try out other architectures and ideas to improve fbeta score


from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop, SGD

#with tpu_strategy.scope():
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 4s 8ms/step - loss: 0.2273 - fbeta: 0.6319 - val_loss: 0.2095 - val_fbeta: 0.6928
Epoch 2/45
506/506 [==============================] - 3s 6ms/step - loss: 0.2008 - fbeta: 0.6768 - val_loss: 0.1875 - val_fbeta: 0.7133
Epoch 3/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1839 - fbeta: 0.7108 - val_loss: 0.1741 - val_fbeta: 0.7183
Epoch 4/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1699 - fbeta: 0.7361 - val_loss: 0.1631 - val_fbeta: 0.7379
Epoch 5/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1619 - fbeta: 0.7513 - val_loss: 0.1620 - val_fbeta: 0.7450
Epoch 6/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1567 - fbeta: 0.7597 - val_loss: 0.1531 - val_fbeta: 0.7786
Epoch 7/45
506/506 [==============================] - 3s 7ms/step - loss: 0.1527 - fbeta: 0.7663 - val_loss: 0.1499 - val_fbeta: 0.7614
Epoch 8/45
506/506 [============================

In [11]:
import gc
gc.collect()

1553

In [12]:
#with tpu_strategy.scope():
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 1s 3ms/step - loss: 0.1817 - fbeta: 0.8190
Test loss: 0.18174028396606445
Test accuracy: 0.8189536929130554


In [13]:
import gc
gc.collect()

741

In [14]:

model.evaluate(x_val, y_val)
print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), 2, average = 'weighted'))

253/253 [==============================] - 1s 3ms/step - loss: 0.1817 - fbeta: 0.8190


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


train fscore:  0.9090358794706274
val fscore:  0.8133871383053753


In [15]:
import gc
del x_train, y_train, x_val, y_val
gc.collect()

1411

In [25]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [17]:
#with tpu_strategy.scope():   
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [00:53<00:00, 381.90it/s]


In [18]:
import gc
gc.collect()

20

In [19]:
#with tpu_strategy.scope():
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 3s


In [20]:
import gc
gc.collect()

741

In [21]:
#converting my encoded labels back to it original form
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)


In [22]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,3.499607e-03,1.767318e-13,8.873148e-06,1.166960e-02,2.445293e-05,9.999306e-01,1.120846e-06,9.931169e-13,2.211383e-04,1.863726e-03,1.072079e-03,4.663793e-07,0.999999,0.002180,1.598846e-04,3.304441e-10,0.001403
1,9.902129e-02,1.774501e-10,3.492414e-03,1.060129e-02,1.367798e-02,9.946027e-01,2.598249e-07,4.473470e-09,1.369907e-01,3.956522e-03,2.902995e-04,6.572056e-03,1.000000,0.033986,2.907227e-02,3.260675e-04,0.170240
2,1.937550e-03,6.547984e-13,2.259073e-12,1.329991e-14,2.524205e-11,3.248095e-14,2.418639e-16,8.752535e-07,3.909072e-06,7.419265e-11,3.587057e-15,1.000000e+00,1.000000,0.000002,3.333629e-11,1.834459e-13,0.040320
3,1.065634e-01,6.901070e-12,2.110123e-03,1.599159e-02,1.509556e-03,2.606965e-01,9.747567e-07,1.937700e-10,2.506021e-01,4.493458e-04,3.186347e-04,9.102114e-01,0.999998,0.010816,9.842344e-03,1.007018e-04,0.025814
4,1.958849e-06,1.525237e-15,2.173509e-02,1.071124e-25,1.504790e-22,4.760524e-08,1.156726e-05,1.038396e-12,1.810191e-09,2.183137e-06,2.995359e-17,9.999996e-01,0.999931,0.001975,6.972645e-13,7.585800e-14,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,2.832972e-08,2.359352e-15,1.761526e-12,2.080061e-19,6.317696e-20,3.668026e-01,1.304417e-02,5.245626e-11,7.365920e-08,1.752009e-03,5.175496e-04,1.635235e-03,0.326188,0.000331,3.606751e-13,2.772944e-21,0.926957
61187,1.220151e-01,2.231107e-11,5.550311e-05,4.062119e-11,3.278861e-15,1.000000e+00,2.712461e-11,1.061929e-07,2.471720e-02,4.025220e-03,7.182975e-07,1.315747e-11,0.999984,0.026739,2.015498e-12,5.143010e-06,0.879140
61188,4.433133e-04,2.597812e-16,8.344912e-07,1.213149e-02,1.055010e-05,9.999990e-01,5.487246e-07,1.131374e-14,3.907297e-05,2.983311e-04,5.715085e-05,7.835013e-09,1.000000,0.000096,3.058586e-05,8.627024e-11,0.001478
61189,1.471180e-04,2.404369e-18,1.992854e-11,3.482270e-23,8.788406e-26,7.270835e-04,9.186201e-01,1.973672e-18,4.492354e-05,4.262559e-05,1.160363e-01,4.179365e-07,0.095785,0.000146,1.262741e-14,2.262043e-16,0.002029


In [23]:
#saving my final result into a csv file
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [02:22<00:00, 429.85it/s]


In [24]:
sam_sub

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,clear cultivation partly_cloudy primary
4,test_4,partly_cloudy primary
...,...,...
61186,file_9995,clear primary water
61187,file_9996,clear primary water
61188,file_9997,clear primary
61189,file_9998,cloudy
